## **Fase 1 - Extract**
*En esta fase inicial, el enfoque está en la extracción de datos brutos directamente desde la plataforma de Airbnb. Utilizando un scraper automatizado, extraemos información esencial sobre las propiedades disponibles, como ubicación, precios, tipo de alojamiento, disponibilidad y reseñas de los usuarios. La extracción es un paso crucial para asegurar que todos los datos relevantes se reúnan y estén listos para su análisis. Nos aseguramos de que los datos se extraigan de forma precisa y consistente, lo cual es fundamental para el éxito de las fases siguientes del proceso. El objetivo de esta fase es consolidar la información extraída en un formato accesible (DataFrame) de manera eficiente y automatizada, para que posteriormente puedan ser limpiados y transformados.* **(necesita edicion - version test)**

In [1]:
# ETL imports
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

# General 
import pandas as pd
import numpy as np
import json          
import os
        

In [ ]:
opts = Options()
opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")

In [ ]:
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=opts)

In [ ]:
driver.get('https://www.airbnb.es/s/Bangkok--Tailandia/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&monthly_start_date=2024-11-01&monthly_length=3&monthly_end_date=2025-02-01&price_filter_input_type=0&channel=EXPLORE&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=autocomplete_click&price_filter_num_nights=28&query=Bangkok%2C%20Tailandia&flexible_trip_dates%5B%5D=march&checkin=2024-10-22&checkout=2024-10-23&zoom_level=12&place_id=ChIJ82ENKDJgHTERIEjiXbIAAQE&location_bb=QV9IIkLJ4HdBV%2BfJQsin1w%3D%3D&flexible_trip_lengths%5B%5D=weekend_trip')

sleep(3)

In [ ]:
titulos_anuncios = driver.find_elements(By.XPATH, '//div[@data-testid="listing-card-title"]') #Se usa corchete para sacar todos los elementos en lista.
for titulo in titulos_anuncios:
    print(titulo.text)